In [1]:
# USAGE
# python detect_glass_video.py

#import the necessary packages
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import argparse
import imutils
import time



In [2]:
 
def detect_and_predict_glass(frame, faceNet, glass):

    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),
        (104.0, 177.0, 123.0))

    faceNet.setInput(blob)
    detections = faceNet.forward()
    #print(detections.shape)

    faces = []
    locs = []
    preds = []
    


    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5: 
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # ensure the bounding boxes fall within the dimensions of
            # the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            # extract the face ROI, convert it from BGR to RGB channel
            # ordering, resize it to 224x224, and preprocess it
            
            face = frame[startY:endY, startX:endX]
            if face is None or face.shape[0] == 0 or face.shape[1] == 0:
                continue
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            # add the face and bounding boxes to their respective
            # lists
            faces.append(face)
            locs.append((startX, startY, endX, endY))

    if len(faces) > 0:
      
        faces = np.array(faces, dtype="float32")
        preds = glass.predict(faces, batch_size=32)

    return (locs, preds)
  




In [3]:
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset", required=True, help="path to input dataset")
ap.add_argument("-p", "--plot", type=str, default="plot.png", help="path to output loss/accuracy plot")
ap.add_argument("-m", "--model", type=str, default="glass_detectorch.model",
help="path to output face glass detector model")
args = {"face": "face_detector","model": "glass_detector.model", "confidence": 0.5}



In [5]:
# load our serialized face detector model from disk

import os
import cv2
face_detector = "face_detector"
prototxtPath = os.path.join(args["face"], "deploy.prototxt")
weightsPath = os.path.join(args["face"], "res10_300x300_ssd_iter_140000.caffemodel")
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

# load the face glass detector model from disk

print("loading face glass detector model...")
glass = load_model("glass_detector.h5")


# initialize the video stream and allow the camera sensor to warm up

print("starting video stream...")
vs = VideoStream(src=0).start()
time.sleep(2.0)


# loop over the frames from the video stream
while True:
    # grab the frame from the threaded video stream and resize it
    # to have a maximum width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=400)

    # detect faces in the frame and determine if they are wearing a
    # face glass or not
    (locs, preds) = detect_and_predict_glass(frame, faceNet, glass)

    # loop over the detected face locations and their corresponding
    # locations
    for (box, pred) in zip(locs, preds):
        (startX, startY, endX, endY) = box
        (Glass, withoutGlass) = pred

        # Determine the base label and color
        label = "Glass" if Glass > withoutGlass else "No Glass"
        color = (0, 255, 0) if label == "Glass" else (0, 0, 255)
        #label = "{}: {:.2f}%".format(label, max(Glass, withoutGlass) * 100)
        
        # Calculate the detailed status message including permission and confidence
        #confidence = max(Glass, withoutGlass) * 100
        
        if(label=="Glass"):
            cv2.putText(frame,"Wearing Glasses ", (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
            
        elif(label=="No Glass"):
            lab="No Eye wear"
            cv2.putText(frame, lab, (startX, startY - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
            
    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the `s` key was pressed, break from the loop
    if key == ord("s"):
        break

# do a bit of cleanup
cv2.destroyAllWindows()

vs.stream.release()


loading face glass detector model...
starting video stream...
1/1 [==============================] - 0s 66ms/step
